<a href="https://colab.research.google.com/github/kimmokyun/python2/blob/master/python_15_holly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 할리스 크롤링하기


In [1]:
### 구글 드라이버 연결

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/python_demo

Mounted at /content/drive
/content/drive/MyDrive/python_demo


# 1. import

In [3]:
from bs4 import BeautifulSoup
import requests

# 2. 작업결과를 저장할 리스트 준비

In [5]:
result = []

# 3. BeautifulSoup 객체 생성하여 파싱

In [27]:
header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.42"}
url = "https://www.hollys.co.kr/store/korea/korStore2.do"

res = requests.get(url, headers = header)
soup = BeautifulSoup(res.text, 'html.parser')
# print(type(soup))

tbody_tag = soup.find('tbody')
#print(type(tbody_tag))

#tr_list_tag = soup.find_all('tr')
#print(len(tr_list_tag))
#print(type(tr_list_tag))
tbody_tr_list_tag = tbody_tag.find_all('tr')
#print(tbody_tr_list_tag)

In [28]:
for store in tbody_tr_list_tag :
  store_td = store.find_all('td')
  store_area = store_td[0].string
  store_name = store_td[1].string
  store_phone = store_td[5].string
  result.append([store_area] + [store_name] + [store_phone])

In [13]:
#참고
['a'] + ['b']

['a', 'b']

In [29]:
result

[['서울 강남구', '강남우리라운지점', '02-566-1002'],
 ['경기 수원시', '수원영통점', '031-202-3356'],
 ['경기 고양시 덕양구', '원흥역점', '031.967.0302'],
 ['서울 양천구', '오목교역점', '02-2062-8405'],
 ['대구 서구', '롯데시네마프리미엄만경관점', '070-7717-2192'],
 ['전남 순천시', '순천신대점', '061-723-5185'],
 ['서울 금천구', '시흥사거리점', '02-804-9226'],
 ['전북 전주시 완산구', '전주효천점', '063-224-5777'],
 ['제주 제주시', '제주도두해안DT점', '064-745-7301'],
 ['부산 북구', '부산화명점', '070-8884-1101']]

# 테이블을 csv파일로 저장

In [35]:
import csv
#field names
fields = ['StoreArea', 'Store_name', 'Store_phone']
with open('./data_crowing/hollys.csv', 'w', newline = '') as f:
  write = csv.writer(f)
  write.writerow(fields)
  write.writerow(result)


In [30]:
!pwd

/content/drive/MyDrive/python_demo


In [34]:
!mkdir data_crowing

In [37]:
%cd data_crowing

/content/drive/MyDrive/python_demo/data_crowing


In [38]:
!ls *.csv

hollys.csv


In [41]:
%cd ..

/content/drive/MyDrive/python_demo


In [43]:
!cat hollys.csv

cat: hollys.csv: No such file or directory


# 6. 기타 : 저장할때 날짜 지정

In [46]:
import datetime
now = datetime.datetime.today()
print(now)
print(now.utcnow())
print(now.year, now.month, now.day, now.hour, now.minute, now.second)

2022-12-12 07:58:01.130999
2022-12-12 07:58:01.134233
2022 12 12 7 58 1


In [47]:
import time
time.strftime('%Y-%m-%d', time.localtime(time.time()))

'2022-12-12'

## 파이선 파일 작성하여 크롤링 작업하기

In [63]:
from bs4 import BeautifulSoup
import requests

import csv
import datetime


def hollys_store():
  result = []
  header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
  url = 'https://m.hollys.co.kr/store/korStore2.do'

  res = requests.get(url, headers = header)
  html = BeautifulSoup(res.text, 'html.parser')
  tag_tbody = html.find('tbody',{'class':'searchResult'})

  print(type(tag_tbody.find_all('tr')))
  for store in tag_tbody.find_all('tr'):          
     if len(store)<=3 :
        break

     store_td = store.find_all('td')    
     store_area = store_td[0].select('div.store_area > a > span.storeArea')[0].string  
     store_name = store_td[1].select('span.storeName > img')
     if(store_name): 
         store_name =  store_name[0]['alt']  
     else:
        store_name = ''
     store_phone = store_td[2].string
     result.append([store_area]+[store_name]+[store_phone])
  return result

def main():    
    result = hollys_store()   
    print('result',result) 
    # 2022-10-10-hollys.csv
    to_now = datetime.datetime.today().date()
    
    # field names  
    fields = ['StoreArea', 'StoreName', 'StorePhone'] 

    with open('./data_crowing/%s-hollys.csv' % to_now, 'w',newline='') as f: 
       write = csv.writer(f)       
       write.writerow(fields) 
       write.writerows(result)     
   
    print('hollys 데이터 저장 완료')
    del result[:]

if __name__ =='__main__':
    main()

<class 'bs4.element.ResultSet'>
result [['강남우리라운지점', '주차', '02-566-1002'], ['수원영통점', '주차', '031-202-3356'], ['원흥역점', '주차', '031.967.0302'], ['오목교역점', '주차', '02-2062-8405'], ['롯데시네마프리미엄만경관점', '주차', '070-7717-2192']]
hollys 데이터 저장 완료


In [64]:
import json
to_now = datetime.datetime.today().date()
with open('./data_crowing/%s-hollys.csv' % to_now, 'r') as f:
   #print(type(list(f.read())))  
   #print(f.readline())
   for element in f :
     #print(element)
     #print(type(element))
     td_list = element.split(',')
     print(td_list[0], td_list[1], td_list[2])
    # break

StoreArea StoreName StorePhone

강남우리라운지점 주차 02-566-1002

수원영통점 주차 031-202-3356

원흥역점 주차 031.967.0302

오목교역점 주차 02-2062-8405

롯데시네마프리미엄만경관점 주차 070-7717-2192

